In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
from apdx_functions import *
from diesel_solver import solve_diesel_cycle, define_empty_variables

In [ ]:
def x_from_P_and_var(pressure, known_var, known_var_value):
    """
    Compute the quality (x) based on one quality-dependent variable (h, s, u, or v)
    and either pressure (P) or temperature (T).
    """

    # Decide the table base and lookup key
    #if known(vars['T']):
    #    table_base = 'Temperature'
    #    lookup_value = vars['T']
    #else:
    table_base = 'Pressure'
    lookup_value = pressure

    var_f = get_apdx_8ab(table_base, table_base[0], lookup_value, known_var + 'f')
    var_g = get_apdx_8ab(table_base, table_base[0], lookup_value, known_var + 'g')

    x = (known_var_value - var_f) / (var_g - var_f)

    return x

x = x_from_P_and_var(0.3, 's', 5)
print(x)
hf, hg = get_apdx_8ab('Pressure', 'P', 0.3, 'hf'), get_apdx_8ab('Pressure', 'P', 0.3, 'hg')
heat = (1-x) * (hg - hf)
print(heat, 'kJ/kg')

In [ ]:
get_apdx_8ab('Pressure', 'P', 0.2, 'hf')

In [ ]:
# Example usage of the apdx_function module:
print("Retrieving APDX 1 data for CO2:")
print(get_apdx_1('CO2', 'R', True))

print("\nRetrieving APDX 7 data for Air:")
print(get_apdx_7('T', 300, 'Pr'))

print("\nRetrieving APDX 8 data for saturated Water:")
print(get_apdx_8ab('Pressure', 'P', 0.50, 'vf'))

print("\nRetrieving APDX 9 data for saturated R134a:")
print(get_apdx_9ab('Pressure', 'P', 0.50, 'vf'))

In [ ]:
print(get_apdx_7('T', 300, 'Pr'))
print(get_apdx_7('Pr', 30.492, 'h'))
print(get_apdx_7('T', 300, 'h'))
print(get_apdx_7('h', 799.71, 'T'))

In [ ]:
print(h_air(1976.4))
print(h_air(988.2))

In [ ]:
# Example usage of the diesel_solver module
variables = define_empty_variables()

variables[0]['r'] = 20  # Compression ratio
variables[0]['rc'] = 2.0  # Cut-off ratio
variables[1]['T1'] = 298.15  # K
variables[1]['P1'] = 100  # kPa

solve_diesel_cycle(variables)

In [ ]:
# Example usage of the diesel_solver module
variables = define_empty_variables()

variables[0]['r'] = 18.0  # Compression ratio
variables[0]['rc'] = 2.0  # Cut-off ratio
variables[1]['T1'] = 300  # K
variables[1]['P1'] = 100  # kPa

solve_diesel_cycle(variables)

In [ ]:
# Example usage of the diesel_solver module
variables = define_empty_variables()

variables[3]['T3'] = 2200 # K
variables[2]['P2'] = 6500 # Cut-off ratio
variables[1]['T1'] = 300  # K
variables[1]['P1'] = 95  # kPa

solve_diesel_cycle(variables)

In [ ]:
# Example usage of the diesel_solver module
variables = define_empty_variables()

variables[3]['T3'] = 1800 # K
variables[1]['P1'] = 95 # Cut-off ratio
variables[1]['T1'] = 290  # K
variables[0]['r'] = 20  # kPa
variables[1]['V1'] = 0.0005 # m^3

solve_diesel_cycle(variables)

In [ ]:
# Load the new uploaded CSV
file_path_new = 'Appendix-data/9c-Superheated-R134a.csv'
df_new = pd.read_csv(file_path_new, header=1)


# Convert Pressure and Temperature columns to numeric
df_new['P'] = pd.to_numeric(df_new['P'], errors='coerce')
df_new['T'] = pd.to_numeric(df_new['T'], errors='coerce')

# Now plot
plt.figure(figsize=(10, 6))
plt.scatter(df_new['P'], df_new['T'], s=10)
plt.title('2D Grid: Pressure vs Temperature (Superheated R134a)')
plt.xlabel('Pressure (MPa)')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.show()

In [ ]:
from scipy.interpolate import griddata

# Define the grid over which we want to interpolate
pressure_vals = np.linspace(df_new['P'].min(), df_new['P'].max(), 200)
temperature_vals = np.linspace(df_new['T'].min(), df_new['T'].max(), 200)

# Create a grid
P_grid, T_grid = np.meshgrid(pressure_vals, temperature_vals)

# Pick a property to interpolate, for example, entropy (s)
# First, ensure entropy is numeric
df_new['s'] = pd.to_numeric(df_new['s'], errors='coerce')

# Prepare points and values
points = np.column_stack((df_new['P'], df_new['T']))
values = df_new['s']

print(griddata(points, values, (0.8, 100), method='linear'))

# Interpolate
S_grid = griddata(points, values, (P_grid, T_grid), method='linear')

# Plot the interpolated surface
plt.figure(figsize=(10, 6))
contour = plt.contourf(P_grid, T_grid, S_grid, levels=50)
plt.colorbar(contour, label='Entropy s (kJ/kg·K)')
plt.scatter(df_new['P'], df_new['T'], s=10)
plt.title('Interpolated Entropy (s) over Pressure-Temperature Grid')
plt.xlabel('Pressure (MPa)')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.show()